In [57]:
import pandas as pd
import folium
import branca.element as be

# 1. 데이터 로드
file_path = "재외공관 정보.xlsx"
try:
    df = pd.read_excel(file_path, sheet_name="Sheet1")

except Exception:
    df = pd.read_excel(file_path, header=1)

In [58]:
df

,Location,소재지,위도(Latitude),경도(Longitude),종류,특수지 근무수당,재외공관 구분표
0,Afghanistan,아프가니스탄,33.939110,67.709953,대사관,가,라
1,Algeria,알제리,28.033886,1.659626,대사관,나,라
2,Angola,앙골라,-11.202692,17.873887,대사관,가,라
3,Argentina,아르헨티나,-38.416097,-63.616672,대사관,NaN,다
4,Australia,호주,-34.529963,149.142783,대사관,NaN,가
...,...,...,...,...,...,...,...
188,Dalian (China),다롄,38.913990,121.614700,출장소,NaN,나
189,Brisbane (Australia),브리즈번,-27.468356,153.024819,출장소,NaN,가
190,Anchorage (United States),앵커리지,61.217576,-149.899678,출장소,NaN,나
191,Hagatna (United States),하갓냐,13.499196,144.777382,출장소,NaN,나


In [59]:
# 2. 지도의 중심 설정 및 초기화
# 대략적인 세계 중심 (혹은 서울)으로 설정
map_center = [37.5665, 126.9780]
m = folium.Map(location=map_center, zoom_start=2)

# 3. 분류 및 아이콘 설정
# 공관 분류(가-라)에 따른 색상 정의
color_mapping = {'가': 'green', '나': 'orange', '다': 'red', '라': 'black'}
# 공관 종류(대사관, 총영사관 등)에 따른 아이콘 정의 (Font Awesome 아이콘 사용)
icon_mapping = {'대사관': 'building-columns', '총영사관': 'building', '대표부': 'house-flag', '분관': 'house', '출장소': 'house'}

# 4. 마커 추가 및 팝업 설정
for index, row in df.iterrows():
    # 팝업에 표시될 정보 (HTML 형식)
    classification_text = f"<b>재외공관 구분표:</b> {row['재외공관 구분표']}" if pd.notna(row['재외공관 구분표']) else "분류 정보 없음"
    hazard_text = f"<b>특수지 근무수당:</b> {row['특수지 근무수당']}" if pd.notna(row['특수지 근무수당']) else f"<b>특수지 근무수당:</b> x"

    
    popup_html = f"""
    <h4>주{row['소재지']}{row['종류']}</h4>
    <p>{classification_text}</p>
    <p>{hazard_text}</p>
    """
    
    # 마커 스타일 설정
    marker_color = color_mapping.get(row['재외공관 구분표'], 'gray')
    icon_name = icon_mapping.get(row['종류'], 'question')
    
    folium.Marker(
        location=[row['위도(Latitude)'], row['경도(Longitude)']],
        popup=folium.Popup(popup_html, max_width=300),
        icon=folium.Icon(color=marker_color, icon=icon_name, prefix='fa')
    ).add_to(m)


legend_html = """
     <div style="position: fixed; 
                 bottom: 50px; left: 50px; width: 200px; max-height: 320px; 
                 border:2px solid grey; z-index:9999; font-size:12px;
                 background-color: white; opacity: 0.9; padding: 10px;
                 overflow-y: auto;">
        <h4 style="margin-top:0; border-bottom: 1px solid #ccc; padding-bottom: 5px;">범례 (Legend)</h4>
        
        <p style="margin-bottom: 5px; font-weight: bold;">[특수지 분류] - 색상</p>
        <div style="margin-left: 10px;">
            <i style="background:green; color:green; border-radius:50%; width:10px; height:10px; display:inline-block; margin-right:5px;"></i> 가 등급 (매우 좋음)<br>
            <i style="background:orange; color:orange; border-radius:50%; width:10px; height:10px; display:inline-block; margin-right:5px;"></i> 나 등급 (좋음)<br>
            <i style="background:red; color:red; border-radius:50%; width:10px; height:10px; display:inline-block; margin-right:5px;"></i> 다 등급 (보통)<br>
            <i style="background:black; color:black; border-radius:50%; width:10px; height:10px; display:inline-block; margin-right:5px;"></i> 라 등급 (나쁨)<br>
        </div>
        
        <p style="margin-bottom: 5px; font-weight: bold; margin-top: 10px; border-top: 1px dashed #eee; padding-top: 5px;">[공관 종류] - 아이콘</p>
        <div style="margin-left: 10px;">
            <i class="fa fa-building-columns" style="color:black; margin-right:5px;"></i> 대사관<br>
            <i class="fa fa-building" style="color:black; margin-right:5px;"></i> 총영사관<br>
            <i class="fa fa-house-flag" style="color:black; margin-right:5px;"></i> 대표부<br>
            <i class="fa fa-house" style="color:black; margin-right:5px;"></i> 분관, 출장소<br>
        </div>
     </div>
"""

# HTML 요소를 지도에 추가
html_element = be.Element(legend_html)
m.get_root().html.add_child(html_element)



# 5. 지도를 HTML 파일로 저장
output_map_path = "재외공관_인터랙티브_지도.html"
m.save(output_map_path)

print(f"\n✅ 인터랙티브 지도가 성공적으로 생성되었습니다: {output_map_path}")
print("이제 해당 파일을 웹 브라우저로 열면 지도를 확인할 수 있습니다.")


✅ 인터랙티브 지도가 성공적으로 생성되었습니다: 재외공관_인터랙티브_지도.html
이제 해당 파일을 웹 브라우저로 열면 지도를 확인할 수 있습니다.
